Эксперименты с text моделями

In [10]:
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Set
from collections import Counter
import Levenshtein
import matplotlib.pyplot as plt
from IPython.core.oinspect import get_encoding
from category_encoders import CatBoostEncoder
from src import future_extract as fe
from src import text_model as tm
import matplotlib as mpl
import seaborn as sns


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
mpl.style.use('ggplot')
sns.set_style('white')
from src import preproces_text as pt
import importlib
importlib.reload(pt)

<module 'src.preproces_text' from '/Users/vladislavdemin/PycharmProjects/BB-E-CUP/src/preproces_text.py'>

In [132]:
path_train = '/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_train.csv'
path_test = '/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_test.csv'

metadata_raw = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/processed_data.csv')
metadata_val = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/processed_datatest.csv')

In [133]:
data_val2 = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_test.csv')

In [130]:
data_raw, data_val = pt.full_preprocess(path_train, path_test, get_encoder='ohe')

Initialized TF-IDF + SVD (fast mode)
Processing DataFrame with 197198 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 1972/1972 [00:48<00:00, 40.83it/s]


Created 100 embedding columns
Final DataFrame shape: (197198, 148)
Processing DataFrame with 22760 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 228/228 [00:05<00:00, 39.12it/s]


Created 100 embedding columns
Final DataFrame shape: (22760, 147)
Processing DataFrame with 197198 rows
Text column: name_rus
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 198/198 [00:06<00:00, 32.41it/s]


Created 100 embedding columns
Final DataFrame shape: (197198, 120)
Processing DataFrame with 22760 rows
Text column: name_rus
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 23/23 [00:00<00:00, 26.34it/s]


Created 100 embedding columns
Final DataFrame shape: (22760, 120)


In [134]:
metadata_raw = metadata_raw.drop(['low_price_flag', 'ItemID', "name_rus","has_name","has_category","high_price_flag", 'id'],axis=1)
metadata_val = metadata_val.drop(['low_price_flag', 'ItemID', "name_rus","has_name","has_category","high_price_flag", 'id'],axis=1)

metadata_raw['resolution'] = data_raw['resolution']

metadata_raw_clean, metadata_val_clean = pt.clean_small_brands(metadata_raw, metadata_val)
metadata_raw_clean_enc, metadata_val_clean_enc = pt.poluchenie_dummy(metadata_raw_clean, metadata_val_clean, data_raw)


In [136]:
data_raw1 = data_raw.drop(['brand_name', 'CommercialTypeName4', 'resolution'], axis=1)
data_val1 = data_val.drop(['brand_name', 'CommercialTypeName4', ], axis=1)

conecting_data_raw = pd.concat([metadata_raw_clean_enc ,data_raw1], axis=1) 
conecting_data_val = pd.concat([metadata_val_clean_enc ,data_val1], axis=1) 

In [137]:
conecting_data_raw = conecting_data_raw.drop('resolution', axis=1)
conecting_data_raw['resolution'] = data_raw['resolution']

In [123]:
conecting_data_raw

,brand_name,CommercialTypeName4,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,comments_published_count,photos_published_count,videos_published_count,...,namerus_embed_91,namerus_embed_92,namerus_embed_93,namerus_embed_94,namerus_embed_95,namerus_embed_96,namerus_embed_97,namerus_embed_98,namerus_embed_99,resolution
0,0.003009,0.001202,6.0,4.0,4.0,3.0,32.0,3.0,6.0,0.0,...,0.008878,0.031000,0.005187,-0.005045,0.005048,0.014858,-0.024484,-0.034824,-0.003355,0
1,0.001068,0.001003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.005316,0.014660,0.002073,-0.014827,0.018512,0.026589,0.019831,0.009437,0.031225,0
2,0.090094,0.000164,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.041034,-0.036070,-0.046307,0.038946,-0.037405,0.111679,0.030324,-0.082830,0.101299,0
3,0.088466,0.000028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.026465,0.018288,0.019070,0.003187,-0.068791,-0.057635,-0.016498,-0.001572,-0.057968,0
4,0.088466,0.000028,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.131194,-0.038963,0.116103,-0.051850,-0.240972,-0.033925,-0.057512,-0.078474,-0.083667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197193,0.019910,0.573606,4.0,1.0,0.0,6.0,60.0,76.0,29.0,9.0,...,0.022600,0.008194,0.032195,-0.031180,-0.009302,0.033963,-0.023104,-0.032739,-0.015229,0
197194,0.059817,0.122101,0.0,0.0,0.0,1.0,31.0,33.0,26.0,12.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
197195,0.250919,0.122101,1.0,1.0,1.0,0.0,27.0,10.0,15.0,0.0,...,0.010806,-0.080817,-0.046403,0.034549,0.042453,-0.030052,-0.032096,0.011569,-0.018326,0
197196,0.108964,0.467145,55.0,30.0,27.0,21.0,72.0,208.0,32.0,1.0,...,0.036636,0.035344,0.002209,0.011807,-0.045729,-0.020324,-0.039951,-0.052103,0.011317,1


In [138]:
X = conecting_data_raw.drop('resolution', axis=1)
y = conecting_data_raw['resolution']  # целевая колонка


# train (70%), test (15%), val (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.28, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)



XGB_model5 = XGBClassifier(
    n_estimators=120,
    max_depth=10,  # Уменьшаем глубину   
    scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])*0.3),
    eval_metric='logloss',
    use_label_encoder=False,
    
    
    
)



In [139]:
X_full = pd.concat([X_train, X_val], axis=0)
y_full = pd.concat([y_train, y_val], axis=0)


In [128]:
X_full = X_full.reset_index(drop=True)
y_full = y_full.reset_index(drop=True)


123997    0
175303    0
54596     0
106861    1
180126    0
         ..
101189    0
95937     0
76714     1
50952     0
51888     0
Name: resolution, Length: 169590, dtype: int64

In [140]:
XGB_model5.fit(X_full, y_full)

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [101]:
threshold = 0.5
y_proba = XGB_model5.predict_proba(X_test)[:, 1]
y_pred = (y_proba >= threshold).astype(int)
accuracy_RFC_model = accuracy_score(y_test, y_pred),
report_RFC_model = classification_report(y_test, y_pred)
print(accuracy_RFC_model)
print(report_RFC_model)

NotFittedError: need to call fit or load_model beforehand

In [23]:
val_prediction_proba = XGB_model5.predict_proba(conecting_data_val)[:, 1]
val_prediction = (val_prediction_proba >= threshold).astype(int)

val_prediction.sum()

np.int64(2273)

In [25]:
submission = pd.DataFrame({
    'id': data_val2['id'],
    'prediction': val_prediction
    
})
submission.to_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv', index=False)
dddd = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv')
dddd['prediction'].sum()

np.int64(2273)